# Lesson 3 Assignment
# Houda Aynaou

## Workplace Scenario
A superstore wants to identify customer groupings that visit their stores based on customer transaction (sales) data. These groupings are used for targeted promotions. You are asked to perform customer segmentation on the provided data.


## To do: 
1. Read dataset
2. Calculate the Recency, Frequency, Monitory for each customer
3. Using K-means algorithm, perform customer segmentation
4. Describe your results

In [1]:
# imports 
import numpy as np
import pandas as pd 

# 1. Read dataset

In [ ]:
data = pd.read_csv()